# Creating Maps with Cartopy

[Documentation](https://scitools.org.uk/cartopy/docs/latest/matplotlib/intro.html)

Cartopy can be easily integrated with matplotlib. It handels the projections and the conversion from projections into other, relatively easy.


### Installation

[Installation documentation](https://scitools.org.uk/cartopy/docs/latest/installing.html)

Via Conda

Dependencies: GEOS, Shapely, and pyshp

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt

import matplotlib.pyplot as plt
import numpy as num

The simplest map plot can be created by calling *plt.axes* with the key word *projection*. The created *axis object* has now new functionalities, e.g. coastlines.

In [ ]:
fig = plt.figure(figsize=(16,9)) 
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

We can add topography (Cartopy by default only supports a really rough one), limit the map to certain areas and add a grid with labels. 

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.stock_img()
ax.set_extent([-10, 30, 30, 70]) # lower, lon, upper lon, lower lat, upper lat
# or with set_xlim and set_ylim
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)


Europe looks a bit odd.

The projection that we used here is *PlateCarree*, which is the *Equirectangular projection*.

Finding the *best* projection is a topic on its own. Cartopy supports several projections ([list of available projections](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#cartopy-projections)) - every single one hast some advantages and disadvantages.

We can try now different projections:

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = plt.axes(projection=ccrs.EuroPP())
ax.coastlines()
ax.stock_img()
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

As said at the beginning, with Cartopy it is relatively easy to plot data. You need to know in which projection our data is stored.

The *projection* key word in plt.axes() is used to define the output projection of the final map, while the *transform* argument, e.g. in plt.plot(), states the projection our data. [Detailed documentation](https://scitools.org.uk/cartopy/docs/latest/tutorials/understanding_transform.html)

In [ ]:
BerlinLon, BerlinLat = 13.41, 52.52

lon = num.linspace(BerlinLon - 30, BerlinLon + 30, 60)
lat = num.linspace(BerlinLat - 30, BerlinLat + 30, 60)
lon2d, lat2d = num.meshgrid(lon, lat)

data = num.sqrt((lon2d - BerlinLon) ** 2 + (lat2d - BerlinLat) ** 2)

plt.figure(figsize=(16, 9))
ax = plt.axes(projection=ccrs.PlateCarree())
# ax = plt.axes(projection=ccrs.RotatedPole(central_rotated_longitude=0, pole_longitude=192.0, pole_latitude=52.0))
ax.set_global()
ax.coastlines()
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

plt.plot(BerlinLon, BerlinLat, '*r', transform=ccrs.Geodetic(), label='Berlin')
plt.text(BerlinLon + 3, BerlinLat - 3, 'Berlin', color='black', horizontalalignment='right', transform=ccrs.Geodetic())

ax.contourf(lon, lat, data, transform=ccrs.PlateCarree())
# ax.contourf(lon, lat, data, transform=ccrs.RotatedPole())
plt.legend()
plt.show()


Besides the topography and coastline, we can add [*features*](https://scitools.org.uk/cartopy/docs/latest/reference/feature.html) like rivers, lakes, ocean, land and even political boarders with Cartopy without any external data.

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = plt.axes(projection=ccrs.Mercator())
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
ax.set_extent([-10, 30, 30, 70]) # lower, lon, upper lon, lower lat, upper lat

ax.add_feature(cfeature.OCEAN, zorder=0)
ax.add_feature(cfeature.LAND, zorder=0, edgecolor='black')
rivers_10m = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m')
ax.add_feature(rivers_10m, facecolor='None', edgecolor='b', zorder=1)
ax.add_feature(cfeature.COASTLINE, linewidth=0.3, zorder=1)
ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=1, zorder=1)
ax.add_feature(cfeature.LAKES, alpha=0.5, zorder=1)

For the last example, we already downloaded the station locations of the GRA network. The map shows those stations in Germany with their station names.

In [ ]:
import pandas as pd

df = pd.read_csv('GRA.txt', sep='|')
# df = df[df['Station'].str.contains('GR')]
df = df[df['Latitude'] > 0]

fig = plt.figure(figsize=(16,9))
# ax = plt.axes(projection=ccrs.PlateCarree())
ax = plt.axes(projection=ccrs.Mercator())
ax.coastlines()
ax.stock_img()
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
ax.set_extent([5, 16, 47, 56]) # lower, lon, upper lon, lower lat, upper lat
ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=1, zorder=1)

lats = df['Latitude']
lons = df['Longitude']

# plt.plot(lons, lats, 'kv', transform=ccrs.Geodetic())

for ii, row in df.iterrows():
    plt.plot(row['Longitude'], row['Latitude'], 'kv', transform=ccrs.Geodetic())
    plt.text(row['Longitude'] + 0.1, row['Latitude'], row['Station'], transform=ccrs.Geodetic())